In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import json

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

import seaborn as sns
sns.set_context('notebook')
sns.set(style="whitegrid", font_scale=1.5)
sns.despine()
sns.set_color_codes()

import matplotlib.pyplot as plt
plt.xlim(0, 1)
plt.ylim(0, None)
from matplotlib.ticker import MaxNLocator
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [20, 5]

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

import vtk
import numpy as np
import nibabel as nib
from six import iteritems
from scipy.stats import norm
from nibabel.affines import apply_affine
from vtk.util import numpy_support as ns
from dipy.tracking.streamline import values_from_volume
from dipy.tracking.metrics import winding

In [2]:
main_folder = '/Users/imag2/Dropbox/IMAG2/MAR/'

post_op = ["1-4-02", "1-4-10", "1-4-13", "1-4-18", "1-4-20_190715", "1-4-21_190506", "1-4-22_2019", "1-4-25",
           "1-4-27_190603", "1-4-29_190708", "1-4-32", "1-4-39", "1-4-40", "1-4-53", "1-4-60"]
pre_op = ["1-4-03", "1-4-04_160425", "1-4-05", "1-4-06", "1-4-07", "1-4-11", "1-4-16", "1-4-19",
          "1-4-20_161017", "1-4-21_161107", "1-4-22_2016", "1-4-26", "1-4-27_170213", "1-4-29_170227",
          "1-4-33", "1-4-37", "1-4-38", "1-4-41", "1-4-44", "1-4-45_180312", "1-4-45_190506", "1-4-48",
          "1-4-49", "1-4-51", "1-4-52", "1-4-55", "1-4-56", "1-4-57", "1-4-64"]

filles = ["1-4-03", "1-4-05", "1-4-11", "1-4-16", "1-4-21_161107", "1-4-27_170213", "1-4-37", "1-4-38" ,
          "1-4-44", "1-4-48", "1-4-52"]
gars = [ "1-4-04_160425", "1-4-06", "1-4-07", "1-4-19", "1-4-20_161017" , "1-4-22_2016", "1-4-26", "1-4-29_170227",
        "1-4-33", "1-4-41", "1-4-45_180312", "1-4-45_190506", "1-4-49", "1-4-51", "1-4-55", "1-4-56", "1-4-57", "1-4-64"]

haute = ["1-4-04_160425", "1-4-06", "1-4-16", "1-4-20_161017", "1-4-27_170213", "1-4-33", "1-4-41", "1-4-44",
         "1-4-45_180312", "1-4-45_190506", "1-4-49", "1-4-51", "1-4-52", "1-4-55", "1-4-64"]
basse = ["1-4-03", "1-4-05", "1-4-07", "1-4-11", "1-4-19", "1-4-21_161107", "1-4-22_2016", "1-4-26",
         "1-4-29_170227", "1-4-37", "1-4-38", "1-4-48", "1-4-56", "1-4-57"]

sans_an_medu = ["1-4-03", "1-4-04_160425", "1-4-05", "1-4-06", "1-4-11", "1-4-16", "1-4-19", "1-4-20_161017",
                "1-4-21_161107", "1-4-22_2016", "1-4-27_170213", "1-4-29_170227", "1-4-33", "1-4-38",
                "1-4-41", "1-4-44", "1-4-48", "1-4-51", "1-4-52", "1-4-55", "1-4-56"]
avec_an_medu = ["1-4-07", "1-4-26", "1-4-45_180312", "1-4-45_190506", "1-4-49", "1-4-57", "1-4-64"]

sans_an_sacr = ["1-4-03", "1-4-04_160425", "1-4-05", "1-4-06", "1-4-11", "1-4-19", "1-4-21_161107",
                "1-4-22_2016", "1-4-26", "1-4-27_170213", "1-4-29_170227", "1-4-48", "1-4-51", "1-4-52", "1-4-56", "1-4-57"]
avec_an_sacr = ["1-4-07", "1-4-16", "1-4-20_161017", "1-4-33", "1-4-38", "1-4-41", "1-4-44", "1-4-45_180312", "1-4-45_190506",
                "1-4-49", "1-4-55", "1-4-64"]

sans_an_tot = ["1-4-03", "1-4-04_160425", "1-4-05", "1-4-06", "1-4-11", "1-4-19", "1-4-21_161107",
               "1-4-22_2016", "1-4-27_170213", "1-4-29_170227", "1-4-48", "1-4-51", "1-4-52", "1-4-56"]
avec_an_tot = ["1-4-07", "1-4-16", "1-4-20_161017", "1-4-26", "1-4-33", "1-4-38", "1-4-41", "1-4-44",
               "1-4-45_180312", "1-4-45_190506", "1-4-49", "1-4-55", "1-4-57", "1-4-64"]

temoins = ["1-2-04", "1-2-05", "1-2-07", "1-2-12", "1-2-14", "1-2-19", "1-2-23", "1-2-28_2016", "1-2-33_2017",
          "1-2-37", "1-2-46", "1-2-51", "1-2-52", "1-2-58", "1-5-02", "1-5-03","1-5-04","1-5-05","1-5-06",
          "1-5-07"]

In [3]:
def load_nii(fname):
    img = nib.load(fname)
    return img.get_fdata(), img.affine

def read_vtk(filename):
    if filename.endswith('xml') or filename.endswith('vtp'):
        polydata_reader = vtk.vtkXMLPolyDataReader()
    else:
        polydata_reader = vtk.vtkPolyDataReader()

    polydata_reader.SetFileName(filename)
    polydata_reader.Update()

    polydata = polydata_reader.GetOutput()

    return vtkpolydata_to_tracts(polydata)

def vtkpolydata_to_tracts(polydata):
    result = {'lines': ns.vtk_to_numpy(polydata.GetLines().GetData()),
              'points': ns.vtk_to_numpy(polydata.GetPoints().GetData()),
              'numberOfLines': polydata.GetNumberOfLines()
             }

    data = {}
    if polydata.GetPointData().GetScalars():
        data['ActiveScalars'] = polydata.GetPointData().GetScalars().GetName()
    if polydata.GetPointData().GetVectors():
        data['ActiveVectors'] = polydata.GetPointData().GetVectors().GetName()
    if polydata.GetPointData().GetTensors():
        data['ActiveTensors'] = polydata.GetPointData().GetTensors().GetName()

    for i in range(polydata.GetPointData().GetNumberOfArrays()):
        array = polydata.GetPointData().GetArray(i)
        np_array = ns.vtk_to_numpy(array)
        if np_array.ndim == 1:
            np_array = np_array.reshape(len(np_array), 1)
        data[polydata.GetPointData().GetArrayName(i)] = np_array

    result['pointData'] = data

    tracts, data = vtkpolydata_dictionary_to_tracts_and_data(result)
    return tracts, data

def vtkpolydata_dictionary_to_tracts_and_data(dictionary):
    dictionary_keys = {'lines', 'points', 'numberOfLines'}
    if not dictionary_keys.issubset(dictionary):
        raise ValueError("Dictionary must have the keys lines and points" + repr(dictionary))

    tract_data = {}
    tracts = []

    lines = np.asarray(dictionary['lines']).squeeze()
    points = dictionary['points']

    actual_line_index = 0
    number_of_tracts = dictionary['numberOfLines']
    original_lines = []
    for _ in range(number_of_tracts):
        tracts.append(points[lines[actual_line_index + 1:actual_line_index + lines[actual_line_index] + 1]])
        original_lines.append(
            np.array(lines[actual_line_index + 1:actual_line_index + lines[actual_line_index] + 1], copy=True))
        actual_line_index += lines[actual_line_index] + 1

    if 'pointData' in dictionary:
        point_data_keys = [it[0] for it in iteritems(dictionary['pointData']) if isinstance(it[1], np.ndarray)]

        for k in point_data_keys:
            array_data = dictionary['pointData'][k]
            if k not in tract_data:
                tract_data[k] = [array_data[f] for f in original_lines]
            else:
                np.vstack(tract_data[k])
                tract_data[k].extend([array_data[f] for f in original_lines[-number_of_tracts:]])

    return tracts, tract_data

In [4]:
def get_hist(bundle, metric, affine):
    def streamlines_mapvolume(streamlines, volume, affine):
        mapping = values_from_volume(volume, streamlines, affine)
        return mapping

    scalar_measurement = streamlines_mapvolume(bundle, metric, affine)
    scalar_measurement = [item for sublist in scalar_measurement for item in sublist]
    return np.asarray(scalar_measurement)

def get_length(bundle):
    def _get_length(tract):
        return ((((tract[1:] - tract[:-1]) ** 2).sum(1)) ** .5).sum()
    tracts_lengths = [_get_length(s) for s in bundle]
    return tracts_lengths

def get_winding(bundle):
    windings = [winding(s) for s in bundle]
    return windings

In [5]:
def save_distr(l5, s1, s2, s3, s4, FA, affine):
    l5_series = pd.Series(get_hist(l5, FA, affine), name="L5")
    s1_series = pd.Series(get_hist(s1, FA, affine), name="S1")
    s2_series = pd.Series(get_hist(s2, FA, affine), name="S2")
    s3_series = pd.Series(get_hist(s3, FA, affine), name="S3")
    s4_series = pd.Series(get_hist(s4, FA, affine), name="S4")
    
    frame = pd.DataFrame(
        {'L5': l5_series,
         'S1': s1_series,
         'S2': s2_series,
         'S3': s3_series,
         'S4': s4_series
        } 
    )
  
    frame.to_csv('/Users/imag2/Desktop/FA_1-4-02.csv')

In [6]:
'''
for file in os.listdir(main_folder):
    current_path = os.path.join(main_folder, file)
    if '.vtk' in current_path.lower() and 'l5' in current_path.lower():
        l5 = read_vtk(current_path)[0]
    elif '.vtk' in current_path.lower() and 's1' in current_path.lower():
        s1 = read_vtk(current_path)[0]
    elif '.vtk' in current_path.lower() and 's2' in current_path.lower():
        s2 = read_vtk(current_path)[0]
    elif '.vtk' in current_path.lower() and 's3' in current_path.lower():
        s3 = read_vtk(current_path)[0]
    elif '.vtk' in current_path.lower() and 's4' in current_path.lower():
        s4 = read_vtk(current_path)[0]
    elif '.nii' in current_path.lower() and 'fa' in current_path.lower():
        FA, affine = load_nii(current_path)
        FA = (FA - np.amin(FA)) / (np.amax(FA) - np.amin(FA))

save_distr(l5, s1, s2, s3, s4, FA, affine)
'''

"\nfor file in os.listdir(main_folder):\n    current_path = os.path.join(main_folder, file)\n    if '.vtk' in current_path.lower() and 'l5' in current_path.lower():\n        l5 = read_vtk(current_path)[0]\n    elif '.vtk' in current_path.lower() and 's1' in current_path.lower():\n        s1 = read_vtk(current_path)[0]\n    elif '.vtk' in current_path.lower() and 's2' in current_path.lower():\n        s2 = read_vtk(current_path)[0]\n    elif '.vtk' in current_path.lower() and 's3' in current_path.lower():\n        s3 = read_vtk(current_path)[0]\n    elif '.vtk' in current_path.lower() and 's4' in current_path.lower():\n        s4 = read_vtk(current_path)[0]\n    elif '.nii' in current_path.lower() and 'fa' in current_path.lower():\n        FA, affine = load_nii(current_path)\n        FA = (FA - np.amin(FA)) / (np.amax(FA) - np.amin(FA))\n\nsave_distr(l5, s1, s2, s3, s4, FA, affine)\n"

In [19]:
df = pd.DataFrame()

p = '/Users/imag2/Dropbox/IMAG2/MAR/hauteur_l5.json'
with open(p, 'r') as f:
    dict_json = json.load(f)

for dirs in os.listdir(main_folder):
    name = dirs
    if name not in temoins and name not in post_op:
        current_dir = os.path.join(main_folder,dirs)
        if os.path.isdir(current_dir):
            for file in os.listdir(current_dir):
                current_path = os.path.abspath(os.path.join(main_folder, current_dir, file))
                if '.vtk' in current_path.lower() and 's4' in current_path.lower():
                    if dict_json[name] is not None:
                        ####length
                        #l5 = pd.Series(get_length(read_vtk(current_path)[0]))
                        #current = pd.DataFrame()
                        #current[name] = l5 / dict_json[name]
                        
                        ####winding
                        l5 = read_vtk(current_path)[0]
                        l5 = pd.Series(np.array(get_winding(l5)) / np.array(get_length(l5)))
                        current = pd.DataFrame()
                        current[name] = l5 / dict_json[name]
                        
                        df = pd.concat([df, current], axis=1)
                    else:
                        raise ValueError('No L5 info: {}'.format(name))

df = df.T
sex =[]
hb = []
mid = []
sacr = []
tot = []
for index, row in df.iterrows():
    if index in filles:
        sex.append('F')
    elif index in gars:
        sex.append('M')
    else:
        sex.append('ND')
    if index in haute:
        hb.append('H')
    elif index in basse:
        hb.append('B')
    else:
        hb.append('ND')
    if index in avec_an_medu:
        mid.append(1)
    elif index in sans_an_medu:
        mid.append(0)
    else:
        mid.append('ND')
    if index in avec_an_sacr:
        sacr.append(1)
    elif index in sans_an_sacr:
        sacr.append(0)
    else:
        sacr.append('ND')
    if index in avec_an_tot:
        tot.append(1)
    elif index in sans_an_tot:
        tot.append(0)
    else:
        tot.append('ND')


df.insert(0, 'AN_TOT', tot)
df.insert(0, 'AN_SACR', sacr)
df.insert(0, 'AN_MID', mid)
df.insert(0, 'H_B', hb)
df.insert(0, 'SEX', sex)

#print(df.iloc[0,:])
#df_f = df[df.H_B == 'H'].drop(columns=['SEX', 'H_B', 'AN_MID','AN_SACR', 'AN_TOT'])
#df_f = df_f.values.flatten()
#df_f = df_f[np.isfinite(df_f)]
#df_m = df[df.H_B == 'B'].drop(columns=['SEX', 'H_B', 'AN_MID','AN_SACR', 'AN_TOT'])
#df_m = df_m.values.flatten()
#df_m = df_m[np.isfinite(df_m)]

#print(np.median(df_f))
#print(np.median(df_m))
#from scipy.stats import mannwhitneyu
#print(mannwhitneyu(df_f, df_m, alternative = 'two-sided', use_continuity=False))
df.to_csv('/Users/imag2/Desktop/s4_ta.csv')
#f, axes = plt.subplots(1, 2)
#sns.violinplot(x=df_f, ax=axes[0], orient='v');
#sns.violinplot(x=df_m, ax=axes[1], orient='v', color='orange');

In [7]:
############ VOLUMES

df = pd.DataFrame()

p = '/Users/imag2/Dropbox/IMAG2/MAR/volumes_l5.json'
with open(p, 'r') as f:
    dict_l5 = json.load(f)
    
p = '/Users/imag2/Dropbox/IMAG2/MAR/volumes.json'
with open(p, 'r') as f:
    dict_vol = json.load(f)

for dirs in os.listdir(main_folder):
    if os.path.isdir(os.path.join(main_folder, dirs)):
        name = dirs
        if name not in temoins and name not in post_op:
            if dict_vol[name] is not None:
                try:
                    l5 = dict_vol[name]['L5']# / dict_l5[name]['L5_VERTEBRA']
                except:
                    l5 = np.nan
                try:
                    s1 = dict_vol[name]['S1']# / dict_l5[name]['L5_VERTEBRA']
                except:
                    s1 = np.nan
                try:
                    s2 = dict_vol[name]['S2']# / dict_l5[name]['L5_VERTEBRA']
                except:
                    s2 = np.nan
                try:
                    s3 = dict_vol[name]['S3']# / dict_l5[name]['L5_VERTEBRA']
                except:
                    s3 = np.nan
                try:
                    s4 = dict_vol[name]['S4']# / dict_l5[name]['L5_VERTEBRA']
                except:
                    s4 = np.nan   
                current = pd.DataFrame()
                current[name] = [l5, s1, s2, s3, s4]

                df = pd.concat([df, current], axis=1)
            else:
                raise ValueError('No info: {}'.format(name))

df = df.T
sex =[]
hb = []
mid = []
sacr = []
tot = []
for index, row in df.iterrows():
    if index in filles:
        sex.append('F')
    elif index in gars:
        sex.append('M')
    else:
        sex.append('ND')
    if index in haute:
        hb.append('H')
    elif index in basse:
        hb.append('B')
    else:
        hb.append('ND')
    if index in avec_an_medu:
        mid.append(1)
    elif index in sans_an_medu:
        mid.append(0)
    else:
        mid.append('ND')
    if index in avec_an_sacr:
        sacr.append(1)
    elif index in sans_an_sacr:
        sacr.append(0)
    else:
        sacr.append('ND')
    if index in avec_an_tot:
        tot.append(1)
    elif index in sans_an_tot:
        tot.append(0)
    else:
        tot.append('ND')


df.insert(0, 'AN_TOT', tot)
df.insert(0, 'AN_SACR', sacr)
df.insert(0, 'AN_MID', mid)
df.insert(0, 'H_B', hb)
df.insert(0, 'SEX', sex)

df.to_csv('/Users/imag2/Desktop/ciao.csv')